<a href="https://colab.research.google.com/github/Theanh130124/pythera_clara/blob/master/nemotron_nano_12b_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install -q transformers accelerate torch bitsandbytes
!pip install -q sentencepiece protobuf
!pip install -q ninja packaging
!pip install -q wheel setuptools

# Quan trọng: Cài đặt mamba-ssm từ source
!pip install -q "mamba-ssm>=2.0.0" --no-build-isolation --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (pyproject.toml) ... done
ERROR: Operation cancelled by user


In [10]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import textwrap

In [11]:
# Config để chạy trên Colab (T4/GPU)
model_name = "nvidia/NVIDIA-Nemotron-Nano-9B-v2"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

# Load model với quantization 4-bit
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
    load_in_4bit=True,  # Quantization để tiết kiệm VRAM
    low_cpu_mem_usage=True
)

print(" Model loaded successfully!")

ImportError: mamba-ssm is required by the Mamba model but cannot be imported

In [ ]:
def chat_with_nemotron(messages, max_tokens=512, temperature=0.6, thinking=True):
    """
    Chat với Nemotron model
    """
    # Thêm system prompt cho reasoning
    system_content = "/think" if thinking else "/no_think"

    full_messages = [{"role": "system", "content": system_content}] + messages

    # Tokenize
    tokenized_chat = tokenizer.apply_chat_template(
        full_messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    # Generate
    with torch.no_grad():
        outputs = model.generate(
            tokenized_chat,
            max_new_tokens=max_tokens,
            temperature=temperature,
            top_p=0.95,
            do_sample=True,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=False)

    # Extract chỉ phần response của assistant
    if "<SPECIAL_11>Assistant" in response:
        response = response.split("<SPECIAL_11>Assistant")[-1]
        if "<SPECIAL_12>" in response:
            response = response.split("<SPECIAL_12>")[0]

    return response.strip()

def print_wrapped(text, width=80):
    """In text với word wrap"""
    print("\n" + "="*50)
    for line in text.split('\n'):
        print(textwrap.fill(line, width=width))
    print("="*50)

#Thinking

In [ ]:
messages = [
    {"role": "user", "content": "Explain why the sky appears blue during daytime?"}
]

response = chat_with_nemotron(messages, thinking=True, max_tokens=400)
print_wrapped(" Reasoning Response:")
print_wrapped(response)

#No Thinking

In [ ]:
messages = [
    {"role": "user", "content": "What is the capital of France?"}
]

response = chat_with_nemotron(messages, thinking=False, max_tokens=200)
print_wrapped("🇫🇷 Direct Answer:")
print_wrapped(response)

In [ ]:
messages = [
    {"role": "user", "content": "Giải thích hiệu ứng nhà kính là gì và tại sao nó quan trọng?"}
]

response = chat_with_nemotron(messages, thinking=True, max_tokens=500)
print_wrapped(" Tiếng Việt - Reasoning:")
print_wrapped(response)

In [ ]:
messages = [
    {"role": "user", "content": "Solve: A train travels 120 km in 2 hours. What is its average speed in m/s?"}
]

response = chat_with_nemotron(messages, thinking=True, max_tokens=400)
print_wrapped("Toán học:")
print_wrapped(response)